<a href="https://colab.research.google.com/github/kartoch/colab-eda/blob/master/98%20-%20Loading%20and%20Saving%20from%20GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.oauth2 import service_account
from google.cloud.storage import client
import io
import pandas as pd
from io import BytesIO
import json
from zipfile import ZipFile

In [0]:
SERVICE_ACCOUNT = json.loads(r"""{
  "type": "service_account",
  "project_id": "[REPLACE WITH YOUR FILE]",
  "privat_sae_key_id": "[REPLACE WITH YOUR FILE]",
  "private_key": "[REPLACE WITH YOUR FILE]",
  "client_email": "[REPLACE WITH YOUR FILE]",
  "client_id": "[REPLACE WITH YOUR FILE]",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "[REPLACE WITH YOUR FILE]"
}""")
BUCKET_DATASETS = "[NAME OF YOUR BUCKET TO READ DATASETS]"
BUCKET_PERSONAL = "[NAME OF YOUR BUCKET TO READ/WITE YOUR DATA (CSV or PARQUET)]"

In [0]:
credentials = service_account.Credentials.from_service_account_info(
    SERVICE_ACCOUNT,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = client.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [0]:
START_YEAR = 2007
END_YEAR = 2019

In [0]:
def download_datasets(year, bucket):
    blob_pathname = "PrixCarburants_annuel_" + str(year) + ".zip"
    blob = bucket.blob(blob_pathname)
    return ZipFile(io.BytesIO(blob.download_as_string()))

def generator_zip_file(client):
    bucket = client.bucket(BUCKET_DATASETS)
    for year in range(START_YEAR,END_YEAR+1):
        zip_ref = download_datasets(year, bucket)    
        [xml_filename] = zip_ref.namelist()
        yield (zip_ref.open(xml_filename),year)
        zip_ref.close()

In [0]:
for f,year in generator_zip_file(client):
    print(f,year)

<zipfile.ZipExtFile name='PrixCarburants_annuel_2007.xml' mode='r' compress_type=deflate> 2007
<zipfile.ZipExtFile name='PrixCarburants_annuel_2008.xml' mode='r' compress_type=deflate> 2008
<zipfile.ZipExtFile name='PrixCarburants_annuel_2009.xml' mode='r' compress_type=deflate> 2009
<zipfile.ZipExtFile name='PrixCarburants_annuel_2010.xml' mode='r' compress_type=deflate> 2010
<zipfile.ZipExtFile name='PrixCarburants_annuel_2011.xml' mode='r' compress_type=deflate> 2011
<zipfile.ZipExtFile name='PrixCarburants_annuel_2012.xml' mode='r' compress_type=deflate> 2012
<zipfile.ZipExtFile name='PrixCarburants_annuel_2013.xml' mode='r' compress_type=deflate> 2013
<zipfile.ZipExtFile name='PrixCarburants_annuel_2014.xml' mode='r' compress_type=deflate> 2014
<zipfile.ZipExtFile name='PrixCarburants_annuel_2015.xml' mode='r' compress_type=deflate> 2015
<zipfile.ZipExtFile name='PrixCarburants_annuel_2016.xml' mode='r' compress_type=deflate> 2016
<zipfile.ZipExtFile name='PrixCarburants_annuel_20

In [0]:
df_test = pd.DataFrame(
    {"col1": [1,2,3],
     "col2": [4,5,6]}
).to_csv(path_or_buf="/tmp/test.csv")

In [0]:
def save_file(local_filename, remote_filename):
    bucket = client.get_bucket(BUCKET_PERSONAL)
    blob = bucket.blob(remote_filename)
    blob.upload_from_filename(local_filename)

In [0]:
save_file("/tmp/test.csv","test.csv")

In [0]:
def download_file(local_filename, remote_filename):
    bucket = client.get_bucket(BUCKET_PERSONAL)
    blob = bucket.blob(remote_filename)
    blob.download_to_filename(local_filename)

In [0]:
download_file("/tmp/test2.csv","test.csv")

In [0]:
import filecmp
assert filecmp.cmp('/tmp/test.csv', '/tmp/test2.csv')